In [ ]:
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn import model_selection, metrics
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
dataset = load_dataset("dl4phys/top_tagging")

### Data Fields
The fields in the dataset have the following meaning:

`E_i`: the energy of jet constituent i,

`PX_i`: the x component of the jet constituent's momentum,

`PY_i`: the y component of the jet constituent's momentum,

`PZ_i`: the z component of the jet constituent's momentum,

`truthE`: the energy of the top-quark,

`truthPX`: the x component of the top quark's momentum, 

`truthPY`: the y component of the top quark's momentum,

`truthPZ`: the z component of the top quark's momentum, 

`ttv`: a flag that indicates which split (train, validation, or test) that a jet belongs to. Redundant since each split is provided as a separate dataset, 

`is_signal_new`: the label for each jet. 
It indicates whether the jet is a top quark signal (1) or QCD background (0).

In [ ]:
dataset = dataset.remove_columns(['truthE', 'truthPX', 'truthPY', 'truthPZ', 'ttv']) #we won’t need the top-quark 4-vector columns and ttv

Now we have each row consists of 4-vectors $(E_i, PX_i, PY_i,PZ_i)$ that correspond to the maximum 200 particles that make up each jet. Also, each jet has been padded with zeros, since most won’t have 200 particles. 

In [ ]:
dataset.set_format("pandas")
train_df, test_df = dataset["train"][:], dataset["test"][:]
train_df.head(5)

In [ ]:
train_df['is_signal_new'].value_counts() # check the number of 0s and 1s in 'is_signal_new' column
train_df.isnull().sum() # check presence of missing data

## K-Means

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
momentum_df = train_df.drop(columns=['is_signal_new'])
momentum_df = momentum_df.filter(regex=r'E_\d+$', axis=1)
momentum_df = momentum_df.iloc[:500, :]
label = kmeans.fit_predict(momentum_df)
print(label)

In [ ]:
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)

for i in u_labels:
    plt.scatter(momentum_df.iloc[label == i , 0] , momentum_df.iloc[label == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()


## Data Loader

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        labels = list(self.dataset['is_signal_new'])
        label = labels[idx] 
        sample_data = list(self.dataset.iloc[idx, :-1])
        data = torch.tensor(sample_data, dtype=torch.float32)
        return data, label

In [ ]:
train_dataset = TabularDataset(train_df)
test_dataset = TabularDataset(test_df)

## Network Architecture

In [ ]:
class LinearBlock(nn.Sequential):
    def __init__(self, in_features, out_features):
        super().__init__(
            nn.Linear(in_features=in_features, out_features=out_features),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=out_features)
        )

class Model(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Model, self).__init__()
        
        self.t_tagger = nn.Sequential(
            nn.BatchNorm1d(num_features=in_channels),
            LinearBlock(in_features=in_channels, out_features=200),
            LinearBlock(in_features=200, out_features=50),
            nn.Linear(in_features=50, out_features=out_channels)
        )
        
    def forward(self, x):
        y = self.t_tagger(x)
        return y

In [ ]:
batch_size = 2
model = Model(in_channels=800, out_channels=1)
model

## Training

In [ ]:
from torch.utils.data import random_split

val_size = int(0.1 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader =  DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader =  DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
val_data, val_label = next(iter(val_dataloader))
model(val_data).shape

In [ ]:
def train_model(train_dataloader, val_dataloader, model, lr=0.01, momentum=0.9, nesterov=False, n_epochs=30):
    train_acc = []
    train_loss = []
    v_acc = []
    v_loss = []
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)

    for epoch in range(1, n_epochs):
        print("-------------\nEpoch {}:\n".format(epoch))

        # Run **training***
        loss, acc = run_epoch(train_dataloader, model.train(), optimizer)
        print('Train loss: {:.6f} | Train accuracy: {:.6f}'.format(loss, acc))
        train_loss.append(loss)
        train_acc.append(acc)

        # Run **validation**
        val_loss, val_acc = run_epoch(val_dataloader, model.eval(), optimizer)
        print('Val loss:   {:.6f} | Val accuracy:   {:.6f}'.format(val_loss, val_acc))
        v_loss.append(val_loss)
        v_acc.append(val_acc)

        # Save model
        # torch.save(model, 't-tagging.pt')
    return val_acc, train_acc, train_loss, v_loss, v_acc

In [ ]:
def run_epoch(dataset, model, optimizer):
    losses = []
    batch_accuracies = []

    # If model is in train mode, use optimizer.
    is_training = model.training
    def compute_accuracy(predictions, y):
        return np.mean(np.equal(predictions.detach().numpy(), y.numpy()))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Iterate through batches
    for data, label in dataset:
        # Grab x and y
        x, y = data[:32], label

        # Get output predictions
        out = model(x)
        
        # Predict and store accuracy
        predictions = torch.argmax(out, dim=1)
        batch_accuracies.append(compute_accuracy(predictions, y))

        # Compute loss
        loss = criterion(out.squeeze(), y.float())
        losses.append(loss.data.item())
        # print(f'loss: {loss}')

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(batch_accuracies)
    return avg_loss, avg_accuracy

In [ ]:
lr = 0.01
momentum = 0.9
n_epochs = 30
train_model(train_dataloader, val_dataloader,  model, lr=lr, momentum=momentum, n_epochs=n_epochs)

## Testing the Model

In [ ]:
loss, accuracy = run_epoch(test_batches, model.eval(), None)
print ("Loss on test set:"  + str(loss) + " Accuracy on test set: " + str(accuracy))